In [25]:
%pip install --upgrade e2b_code_interpreter cohere==5.5.3

Note: you may need to restart the kernel to use updated packages.


In [26]:
# Instantiate the Cohere client

import cohere
import os

import json

import os
from dotenv import load_dotenv
load_dotenv()

COHERE_API_KEY = os.environ["COHERE_API_KEY"]
E2B_API_KEY = os.getenv("E2B_API_KEY")

co = cohere.Client(api_key=COHERE_API_KEY)

In [27]:
# Define the tools


def list_calendar_events(date: str):
  events = '[{"start": "14:00", "end": "15:00"}, {"start": "15:00", "end": "16:00"}, {"start": "17:00", "end": "18:00"}]'
  print(f"Listing events: {events}")
  return events

def create_calendar_event(date: str, time: str, duration: int):
  print(f"Creating a {duration} hour long event at {time} on {date}")
  return True


def code_interpret(e2b_code_interpreter, code):
  print("Running code interpreter...")
  exec = e2b_code_interpreter.notebook.exec_cell(
    code,
    # Stream stdout and stderr from the Code Interpreter
    on_stdout=lambda stdout: print("[Code Interpreter]", stdout),
    on_stderr=lambda stderr: print("[Code Interpreter]", stderr),
    # You can also stream code execution results
    # on_result=...
  )

  if exec.error:
    print("[Code Interpreter ERROR]", exec.error)
  else:
    return exec

list_calendar_events_tool = {
  "name": "list_calendar_events",
  "description": "returns a list of calendar events for the specified date, including the start time and end time for each event",
  "parameter_definitions": {
    "date": {
      "description": "the date to list events for, formatted as mm/dd/yy",
      "type": "str",
      "required": True
    }
  }
}

create_calendar_event_tool = {
  "name": "create_calendar_event_tool",
  "description": "creates a calendar event of the specified duration at the specified time and date",
  "parameter_definitions": {
    "date": {
      "description": "the date on which the event starts, formatted as mm/dd/yy",
      "type": "str",
      "required": True
    },
    "time": {
      "description": "the time of the event, formatted using 24h military time formatting",
      "type": "str",
      "required": True
    },
    "duration": {
      "description": "the number of hours the event lasts for",
      "type": "float",
      "required": True
    }
  }
}

execute_python_tool =   {
  "name": "execute_python",
  "description": "Execute python code in a Jupyter notebook cell and returns any result, stdout, stderr, display_data, and error.",
  "parameter_definitions": {
      "code": {
          "description": "The python code to execute in a single cell.",
          "type": "str",
          "required": True
    }
  },
}


# helper function for routing to the correct tool
def invoke_tool(tool_call: cohere.ToolCall):
  if tool_call.name == list_calendar_events_tool["name"]:
    date = tool_call.parameters["date"]
    return [{
        "events": list_calendar_events(date)
    }]
  elif tool_call.name == create_calendar_event_tool["name"]:
    date = tool_call.parameters["date"]
    time = tool_call.parameters["time"]
    duration = tool_call.parameters["duration"]

    return [{
        "is_success": create_calendar_event(date, time, duration)
    }]
  elif tool_call.name == execute_python_tool["name"]:
    code = tool_call.parameters["code"]

    return [{
        "is_success": code_interpret(code)
    }]
  else:
    raise f"Unknown tool name '{tool_call.name}'"

In [28]:
# Check what tools the model wants to use and how to use them
res = co.chat(
    model="command-r-plus",
    preamble="Today is Thursday, may 23, 2024",
    message="Book an hour long appointment for the first available free slot after 3pm. Create a chart with timeline showing the events in calendar.",
    force_single_step=False,
    tools=[list_calendar_events_tool, create_calendar_event_tool, execute_python_tool])

while res.tool_calls:
  print(res.text) # This will be an observation and a plan with next steps

  # invoke the recommended tools
  tool_results = []
  for call in res.tool_calls:
    tool_results.append({"call": call, "outputs": invoke_tool(call)})

  # send back the tool results
  res = co.chat(
    model="command-r-plus",
    chat_history=res.chat_history,
    message="",
    force_single_step=False,
    tools=[list_calendar_events_tool, create_calendar_event_tool, execute_python_tool],
    tool_results=tool_results,
  )

print(res.text) # print the final answer

First, I will check the user's calendar for the day to find the first available free slot after 3pm. Then, I will book an hour-long appointment in that slot. Finally, I will create a chart with a timeline showing the events in the user's calendar.
Listing events: [{"start": "14:00", "end": "15:00"}, {"start": "15:00", "end": "16:00"}, {"start": "17:00", "end": "18:00"}]
The user's calendar shows that they have events from 2pm to 4pm, and then another event from 5pm to 6pm. The first available free slot after 3pm is from 4pm to 5pm. I will now book an hour-long appointment in that slot.
Creating a 1 hour long event at 16:00 on 05/23/2024
I've booked an hour-long appointment for you from 4pm to 5pm today. Here's a chart with a timeline showing the events in your calendar:

| Time | Event |
| --- | --- |
| 2pm to 3pm | Event |
| 3pm to 4pm | Event |
| 4pm to 5pm | Hour-long appointment |
| 5pm to 6pm | Event |
